In [ ]:
from roboqueue import StationaryTarget, TargetList, Telescope, Conditions, TelescopeSchedulingEnv
from spacerocks.utils import time_handler
import numpy as np
from tabulate import tabulate

In [ ]:
telescope = Telescope(obscode='W84', filter='r')
conditions = Conditions(seeing=1, moon=7)

In [ ]:
star_positions = [( 24.42813204, -57.23666007), 
                  ( 68.98000195,  16.50976164),
                  ( 78.63446353,  -8.20163919), 
                  ( 79.17206517,  45.99902927),
                  ( 88.79287161,   7.40703634), 
                  ( 95.98787763, -52.69571799),
                  (101.28854105, -16.71314306), 
                  (114.82724194,   5.22750767),
                  (186.64975585, -63.09905586), 
                  (210.95601898, -60.3729784 ),
                  (219.92041034, -60.83514707)]

# convert to radians    
star_positions = [(np.radians(ra), np.radians(dec)) for ra, dec in star_positions]
print(star_positions)

In [ ]:
targets = TargetList()

for star in star_positions:
    target = StationaryTarget(ra=star[0], dec=star[1], mag=1, required_snr=1_000_000)
    targets.add_target(target)


In [ ]:
epoch = time_handler('12 February 2024').utc.jd[0]

In [ ]:
ra, dec, alt, az, exptime = targets.at(epoch=epoch, telescope=telescope, conditions=conditions)
print(tabulate(zip(ra, dec, alt, az, exptime), headers=['RA', 'DEC', 'ALT', 'AZ', 'EXPTIME'], tablefmt='pretty'))
sky_state = np.array([ra, dec, alt, az, exptime]).T


In [ ]:
# Tune the reward weights
reward_config = { 'exptime': 1, 'n_observed_targets': 1 }

# Create training environment
max_duration = 24 * 3600        # [s] fix it later
env = TelescopeSchedulingEnv(telescope, conditions, max_duration, sky_state, targets, reward_config)

# Reset the environment and get the initial observation
observation = env.reset()

# Select an action based on the current observation
# TODO: replace with PPO agent 
while not done:
    action = agent.select_action(observation)

    # Take a step in the environment using the selected action
    observation, reward, done, info = env.step(action)

In [ ]:
np.array([exptime, alt]).T